# 1. 确认GPU

In [1]:
!nvidia-smi

Mon Jul 21 16:25:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.49                 Driver Version: 576.49         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   42C    P0             17W /   80W |       0MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
from torch import nn
print(torch.__version__)
print(torch.cuda.is_available())

torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:0')

2.7.0+cu118
True


(device(type='cpu'),
 <torch.cuda.device at 0x2430f1f4510>)

In [3]:
print(torch.cuda.device_count())

1


In [9]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回gpu"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]。"""
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]      
    return devices if devices else [torch.device('cpu')]
                    
print(try_gpu())
print(try_gpu(10))
print(try_all_gpus())

cuda:0
cpu
[device(type='cuda', index=0)]


In [10]:
# 查询张量所在设备
X = torch.tensor([1,2,3])
print(X.device) # 默认在CPU内存上

cpu


In [11]:
# 存储在GPU上
X = torch.ones(2,3,device=try_gpu())
X 

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [12]:
# 在第二个GPU上创建一个随机张量
Y = torch.rand(2,3,device=try_gpu(1))
print(Y.device) # 没有1号GPU，则放到CPU上
Y

cpu


tensor([[0.9438, 0.0918, 0.2306],
        [0.0824, 0.8068, 0.0367]])

In [13]:
# 要计算X+Y，我们需要决定在哪里执行这个操作
Z = X.cuda(0) # X+Y必须X和Y都在同一个GPU上
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [14]:
Y = torch.rand(2,3,device=try_gpu(0))
print(Y + Z)
# .cuda() 是一个张量方法，它会将张量移动到 GPU 上
# is 是 Python 中的一个身份比较运算符。它用来检查两个变量是否引用了内存中的同一个对象。
# 这与 == 运算符不同，== 检查的是两个对象的值是否相等。
# * 将张量 Z 移动到 CUDA 设备 0 后返回的张量，是否和原始张量 Z 是内存中的同一个对象？
Z.cuda(0) is Z # 如果变量在0号GPU时，就返回True

tensor([[1.5696, 1.2940, 1.1538],
        [1.6253, 1.4972, 1.4800]], device='cuda:0')


True

In [ ]:
# 神经网络与GPU
net = nn.Sequential(nn.Linear(3,1)) # 创建神经网络时已经把权重初始化好了

net = net.to(device=try_gpu()) # ! 把所有参数在0号GPU上拷贝一份
X = torch.ones(2,3,device=try_gpu()) # X 在0号GPU上
net(X) # 所以前项运算所有元素都在0号GPU上运行

tensor([[0.7260],
        [0.7260]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [16]:
# 确认模型参数存储在同一个GPU上
net[0].weight.data.device

device(type='cuda', index=0)